<a href="https://colab.research.google.com/github/denam98/CentimentAnnalysisModel/blob/main/FinalProject_Task_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np

In [4]:
data_set = pd.read_csv("/content/drive/MyDrive/DataSets/Tweets.csv", encoding='latin-1')
data_set.head(5)
data_set.shape

(14640, 15)

In [5]:
data_set = data_set[["airline_sentiment", "text"]]
data_set.head(5)

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


# Preprocessing text

In [6]:
import nltk

import string

import re


from nltk.stem.porter import PorterStemmer


nltk.download('stopwords')

from nltk.corpus import stopwords


nltk.download('punkt')

ps = PorterStemmer()



def clean_text(text):

    text = text.lower()

    text = re.sub(r'http.?://[^\s]+[\s]?', '', text)

    text = nltk.word_tokenize(text)

    y = []

    for i in text:
      if i not in stopwords.words('english'):
            y.append(i)

    text = y[:]

    y.clear()

    for i in text:
        y.append(ps.stem(i))

    return " ".join(y)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
data_set['text_cleaned'] = data_set['text'].apply(clean_text)
data_set.head(5)
data_set.groupby('airline_sentiment').describe()

text         \
                  count unique   
airline_sentiment                
negative           9178   9087   
neutral            3099   3067   
positive           2363   2298   

                                                                           \
                                                                 top freq   
airline_sentiment                                                           
negative           @AmericanAir that's 16+ extra hours of travel ...    2   
neutral                                           @SouthwestAir sent    5   
positive                                            @JetBlue thanks!    5   

                  text_cleaned         \
                         count unique   
airline_sentiment                       
negative                  9178   9083   
neutral                   3099   3027   
positive                  2363   2262   

                                                                           
                                                                 top freq  
airline_sentiment                                                          
negative           @ americanair fyi ... call still get drop . ho...    2  
neutral            @ jetblu 's ceo battl appeas passeng wall stre...    8  
positive                                            @ jetblu thank !   12

# Feature Extraction

In [8]:
import sklearn.model_selection as sk
from sklearn.feature_extraction.text import TfidfVectorizer

In [9]:
tf_vec = TfidfVectorizer(max_features=3000)
X = tf_vec.fit_transform(data_set['text_cleaned']).toarray()
X.shape

(14640, 3000)

In [10]:
# x = data_set['text_cleaned'].values
y = data_set['airline_sentiment'].values

In [11]:
x_train, x_test, y_train, y_test = sk.train_test_split(X, y, test_size=.2, random_state=2)
x_train.shape

(11712, 3000)

# Training a multinomial Naïve Bayes classifier to predic the sentiment of the tweet

In [14]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

clf = MultinomialNB()
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(accuracy_score(y_test,y_pred))
# print(clf.predict(X[2:3]))

0.7219945355191257


# Training a Random Forest classifier to predict the sentiment and to find accuracy of the model

In [15]:
model = RandomForestClassifier()
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
print(accuracy_score(y_test,y_pred))

0.7506830601092896
